# DISNEY DATASET

# Scrape & clean a list of disney wikipedia pages to create a dataset to further analyze

### Get info Box - store in python dictionary
>Build for single disney movie 

In [1]:
from bs4 import BeautifulSoup
import requests
import json
import re
from datetime import datetime
import urllib.parse
from dotenv import load_dotenv
import os
import requests
import urllib
import random
import time
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/Cars_3"

header = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36"
}

r = requests.get(url, headers=header)
soup = BeautifulSoup(r.content, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Cars 3 - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled 

In [3]:
info_box = soup.find("table", class_="infobox vevent")
info_rows = info_box.find_all("tr")

for row in info_rows:
    print(row.prettify())

<tr>
 <th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
  Cars 3
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <span class="mw-default-size" typeof="mw:File/Frameless">
   <a class="mw-file-description" href="/wiki/File:Cars_3_poster.jpg" title='Lightning McQueen, Cruz Ramirez and Jackson Storm race along a beach. "Cars 3" is written in the bottom right corner, between the release date "June 16" on the bottom.'>
    <img alt='Lightning McQueen, Cruz Ramirez and Jackson Storm race along a beach. "Cars 3" is written in the bottom right corner, between the release date "June 16" on the bottom.' class="mw-file-element" data-file-height="370" data-file-width="250" decoding="async" height="370" src="//upload.wikimedia.org/wikipedia/en/9/94/Cars_3_poster.jpg" width="250"/>
   </a>
  </span>
  <div class="infobox-caption">
   Theatrical release poster
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space: 

In [4]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

movie_info = {}

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info["title"] = row.find("th").get_text(" ", strip=True)
    elif index == 1:
        continue
    else:
        header = row.find("th").get_text(" ", strip=True)
        value = get_content_value(row.find("td"))
        movie_info[header] = value

movie_info

{'title': 'Cars 3',
 'Directed by': 'Brian Fee',
 'Screenplay by': ['Kiel Murray', 'Bob Peterson', 'Mike Rich [ 1 ]'],
 'Story by': ['Brian Fee',
  'Ben Queen',
  'Eyal Podell',
  'Jonathan E. Stewart [ 1 ]'],
 'Produced by': 'Kevin Reher',
 'Starring': ['Owen Wilson',
  'Cristela Alonzo',
  'Chris Cooper',
  'Armie Hammer',
  'Larry the Cable Guy',
  'Bonnie Hunt',
  'Nathan Fillion',
  'Lea DeLaria',
  'Kerry Washington'],
 'Cinematography': ['Jeremy Lasky (camera)', 'Kim White (lighting) [ 1 ]'],
 'Edited by': 'Jason Hudak',
 'Music by': 'Randy Newman [ 2 ]',
 'Production company': 'Pixar Animation Studios',
 'Distributed by': 'Walt Disney Studios Motion Pictures [ a ]',
 'Release dates': ['May 23, 2017 ( 2017-05-23 ) ( Kannapolis )',
  'June 16, 2017 ( 2017-06-16 ) (United States)'],
 'Running time': '102 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$175 million [ 3 ]',
 'Box office': '$383.9 million [ 4 ]'}

### Get info box for all movies 
>implementation based of a single solution

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films"

header = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36"
}

r = requests.get(url, headers=header)
soup = BeautifulSoup(r.content, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Walt Disney Pictures films - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-li

In [6]:
movies = soup.select(".wikitable.sortable i a")
movies = [movie["href"] for movie in movies]
movies[:10]

['/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)',
 '/wiki/Pinocchio_(1940_film)',
 '/wiki/Fantasia_(1940_film)',
 '/wiki/The_Reluctant_Dragon_(1941_film)',
 '/wiki/Dumbo',
 '/wiki/Bambi',
 '/wiki/Saludos_Amigos',
 '/wiki/Victory_Through_Air_Power_(film)',
 '/wiki/The_Three_Caballeros',
 '/wiki/Make_Mine_Music']

In [7]:
def get_content_value(row_data):
    if row_data.find("li"):
        values = [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        values = [text for text in row_data.stripped_strings]
    else:
        values = [row_data.get_text(" ", strip=True).replace("\xa0", " ")]
        
    cleaned = [v.strip() for v in values if v]

    return cleaned

    

def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()
        
def get_title_tag(soup):
    title_tag = soup.find("th", class_="infobox-above")
    if title_tag:
        return title_tag.get_text(" ", strip=True)
    return None
    
    

def get_box_info(url):

    header = {
        "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36"
    }

    r = requests.get(url, headers=header)
    
    soup = BeautifulSoup(r.content, 'html.parser')
    clean_tags(soup)

    info_box = soup.find("table", class_="infobox")
    if info_box is None:
        print(f"No infobox found for {url}")
        return {}

    info_rows = info_box.find_all("tr")
    movie_info = {}

    for index, row in enumerate(info_rows):
        if index == 0:
            title_th = row.find("th")
            if title_th:
                movie_info["title"] = title_th.get_text(" ", strip=True)
            continue

        header = row.find("th")
        if header:
            content_key = header.get_text(" ", strip=True)
            td = row.find("td")

            if td is None:
                continue

            content_value = get_content_value(td)
            movie_info[content_key] = content_value if content_value else None

    return movie_info

In [8]:
get_box_info("https://en.wikipedia.org/wiki/Shipwrecked_(1990_film)")

{'title': 'Shipwrecked',
 'Directed by': ['Nils Gaup'],
 'Written by': ['Nils Gaup', 'Bob Foss', 'Greg Dinner', 'Nick Thiel'],
 'Based on': ['Haakon Haakonsen: En Norsk Robinson', 'by', 'Oluf Falck-Ytter'],
 'Produced by': ['John M. Jacobsen', 'Nigel Wooll'],
 'Starring': ['Stian Smestad', 'Gabriel Byrne'],
 'Cinematography': ['Erling Thurmann-Andersen'],
 'Edited by': ['Nils Pagh Andersen'],
 'Music by': ['Patrick Doyle'],
 'Production companies': ['Walt Disney Pictures', 'AB Svensk Filmindustri'],
 'Distributed by': ['Buena Vista Pictures Distribution'],
 'Release dates': ['3 October 1990'],
 'Running time': ['93 minutes'],
 'Countries': ['Norway', 'Sweden', 'United States'],
 'Languages': ['Norwegian', 'English'],
 'Budget': ['60 million Norwegian Kroner (around $8.7 million in 1989)'],
 'Box office': ['$17 million (Norway/USA)']}

In [10]:
url = "https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films"

header = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36"
}

r = requests.get(url, headers=header)
soup = BeautifulSoup(r.content, 'html.parser')
tables = soup.select("table.wikitable.sortable")

bad_sections = ["Upcoming", "Undated films", "In development"]
filtered_tables = []

for table in tables:
    section = table.find_previous(["h2", "h3"])
    if section and any(bad.lower() in section.get_text(strip=True).lower() for bad in bad_sections):
        continue  # pomiń
    filtered_tables.append(table)

movies = []
for table in filtered_tables:
    movies.extend(table.select("i a[href^='/wiki/']"))

movie_info_list = []
base_url = "https://en.wikipedia.org/"

for index, movie in enumerate(movies):
    if index % 10 == 0:
        print(index)
    try:
        relative_path = movie["href"]
        full_url = base_url + relative_path
        title = movie["title"]
        
        movie_info_list.append(get_box_info(full_url))
    except Exception as e:
        print(movie.get_text())
        print(e)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530


In [11]:
len(movie_info_list)

536

### Save/ Reload Movie Data

In [12]:
def save_data(title, data):
    with open(title, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [13]:
save_data("disney_data_scraped.json", movie_info_list)

### Task 3: Clean our data

In [14]:
movie_info_list = load_data("disney_data_scraped.json")

### Subtasks

- ~~Clean up references [1]~~
- Convert dates into datetime object  
- ~~Convert running time into an integer~~  
- ~~Split up the long strings~~  
- ~~Convert Budget & Box office to numbers~~

In [15]:
movie_info_list[-10]

{'title': 'Blink',
 'Directed by': ['Edmund Stenson', 'Daniel Roher'],
 'Produced by': ['Melanie Miller', 'Diane Becker'],
 'Cinematography': ['Jean-Sébastien Francoeur'],
 'Edited by': ['Ryan Mullins', 'Miranda Yousef'],
 'Music by': ['Tamar-kali'],
 'Production companies': ['National Geographic Documentary Films',
  'Fishbowl Films',
  'MRC',
  'Eyesteel Film'],
 'Distributed by': ['National Geographic Documentary Films (Worldwide)',
  'Walt Disney Studios Motion Pictures',
  '(United States and Canada)'],
 'Release dates': ['August 30, 2024 ( Telluride )', 'October 4, 2024'],
 'Running time': ['84 minutes'],
 'Country': ['United States'],
 'Language': ['English']}

In [16]:
[movie.get("Running time", "N/A") for movie in movie_info_list]
# min, list of values, 22-24 minutes (we take avg ?), mins, N/A, 68\nminutes

[['83 minutes'],
 ['88 minutes'],
 ['126 minutes'],
 ['74 minutes'],
 ['64 minutes'],
 ['70 minutes'],
 ['42 minutes'],
 ['65 min'],
 ['71 minutes'],
 ['75 minutes'],
 ['94 minutes'],
 ['73 minutes'],
 ['75 minutes'],
 ['82 minutes'],
 ['68 minutes'],
 ['76 minutes'],
 ['96 minutes'],
 ['75 minutes'],
 ['84 minutes'],
 ['77 minutes'],
 ['92 minutes'],
 ['69 minutes'],
 ['81 minutes'],
 ['60 minutes (VHS and Wild Discovery version)', '71 minutes (original)'],
 ['127 minutes'],
 ['93 minutes'],
 ['76 minutes'],
 ['75 minutes'],
 ['73 minutes'],
 ['85 minutes'],
 ['81 minutes'],
 ['70 minutes'],
 ['90 minutes'],
 ['80 minutes'],
 ['75 minutes'],
 ['84 minutes'],
 ['83 minutes'],
 ['72 minutes'],
 ['97 minutes'],
 ['75 minutes'],
 ['104 minutes'],
 ['93 minutes'],
 ['105 minutes'],
 ['95 minutes'],
 ['97 minutes'],
 ['134 minutes'],
 ['22–24 minutes'],
 ['69 minutes'],
 ['92 minutes'],
 ['126 minutes'],
 ['79 minutes'],
 ['97 minutes'],
 ['128 minutes'],
 ['73 minutes'],
 ['91 minutes'],
 

In [17]:
# Convert running time into an integer
def minutes_to_integer(running_time):
    if running_time == "N/A":
        return None
    
    if isinstance(running_time, list): 
        value_string = running_time[0]
    else:
        value_string = running_time
    
    try:
        if ' ' in value_string:
            number_part = value_string.split(' ')[0]
        else:
             number_part = value_string.split('\n')[0]
            
        final_value_string = number_part.split("–")[0].split("-")[0]
        
        return int(final_value_string)
    
    except ValueError:
        return None

for movie in movie_info_list:
    movie["Running time (int)"] = minutes_to_integer(movie.get("Running time") or "N/A")

In [18]:
movie_info_list[-30]

{'title': 'Hocus Pocus 2',
 'Directed by': ['Anne Fletcher'],
 'Screenplay by': ["Jen D'Angelo"],
 'Based on': ['Mick Garris', 'David Kirschner'],
 'Produced by': ['Lynn Harris'],
 'Starring': ['Bette Midler',
  'Sarah Jessica Parker',
  'Kathy Najimy',
  'Sam Richardson',
  'Doug Jones',
  'Whitney Peak',
  'Belissa Escobedo',
  'Tony Hale',
  'Hannah Waddingham'],
 'Cinematography': ['Elliot Davis'],
 'Edited by': ['Julia Wong'],
 'Music by': ['John Debney'],
 'Production companies': ['Walt Disney Pictures',
  'David Kirschner Productions',
  'Weimaraner Republic Pictures'],
 'Distributed by': ['Disney+'],
 'Release date': ['September 30, 2022'],
 'Running time': ['104 minutes'],
 'Country': ['United States'],
 'Language': ['English'],
 'Budget': ['$40 million'],
 'Running time (int)': 104}

In [19]:
print([movie.get("Running time (int)") or "N/A" for movie in movie_info_list])

[83, 88, 126, 74, 64, 70, 42, 65, 71, 75, 94, 73, 75, 82, 68, 76, 96, 75, 84, 77, 92, 69, 81, 60, 127, 93, 76, 75, 73, 85, 81, 70, 90, 80, 75, 84, 83, 72, 97, 75, 104, 93, 105, 95, 97, 134, 22, 69, 92, 126, 79, 97, 128, 73, 91, 105, 98, 130, 89, 93, 67, 98, 100, 118, 103, 110, 80, 79, 91, 91, 97, 118, 139, 131, 92, 87, 116, 93, 114, 110, 131, 101, 110, 84, 78, 75, 164, 106, 110, 99, 113, 108, 102, 85, 91, 93, 100, 100, 79, 96, 113, 89, 118, 92, 88, 92, 87, 93, 93, 93, 90, 83, 96, 88, 89, 91, 93, 92, 97, 100, 100, 89, 'N/A', 91, 112, 115, 95, 91, 97, 104, 74, 77, 104, 129, 101, 94, 104, 90, 100, 88, 93, 98, 112, 84, 97, 97, 114, 96, 97, 109, 83, 90, 107, 96, 103, 91, 95, 105, 113, 80, 101, 90, 74, 90, 89, 110, 74, 93, 84, 83, 73, 77, 107, 93, 88, 108, 84, 121, 89, 104, 90, 86, 84, 108, 107, 96, 98, 105, 109, 94, 106, 102, 88, 102, 102, 97, 111, 97, 96, 96, 78, 81, 108, 89, 100, 89, 81, 92, 99, 89, 79, 91, 101, 104, 103, 86, 106, 74, 93, 92, 98, 76, 95, 93, 87, 93, 87, 128, 77, 88, 86, 9

In [20]:
[movie.get("Budget") or "N/A" for movie in movie_info_list]
# $1.5 million, N/A, under $1 million or $1,250,000, $3.6–4 million, AU$1 million, A$8.7 million, 60 million Norwegian Kroner, $4,000,000, ¥', '3.4 billion', '(', 'US$', '34 million), €57.1 million, ['$410.6 million', '$378.5 million'],

[['$1.5 million'],
 ['$2.6 million'],
 ['$2.28 million'],
 ['$600,000'],
 ['$950,000'],
 ['$2 million'],
 'N/A',
 ['$788,000'],
 'N/A',
 ['$1.35 million'],
 ['$2.125 million'],
 'N/A',
 ['$1.5 million'],
 ['$1.5 million'],
 'N/A',
 ['$2.2 million'],
 ['$1.8 million'],
 ['$3 million'],
 'N/A',
 ['$4 million'],
 ['$2 million'],
 ['$300,000'],
 ['$1.8 million'],
 'N/A',
 ['$5 million'],
 ['$7 million'],
 ['$4 million'],
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 ['$700,000'],
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 ['$6 million'],
 ['under $1 million or $1,250,000'],
 'N/A',
 ['$2 million'],
 'N/A',
 'N/A',
 ['$2.5 million'],
 'N/A',
 'N/A',
 'N/A',
 ['$5 million'],
 ['$3.6–4 million'],
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 ['$3 million'],
 'N/A',
 ['$3 million'],
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 ['$2 million'],
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 ['$3 million'],
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 ['$4.4–6 million'],
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 'N

In [21]:
[movie.get("Box office") or "N/A" for movie in movie_info_list]

[['$418 million'],
 ['$164 million'],
 ['$76.4–$83.3 million (United States and Canada)'],
 ['$960,000 (worldwide rentals)'],
 ['>$1.3 million (est. United States/Canada rentals, 1941)'],
 ['$267.4 million'],
 ['$1 million (worldwide rentals)'],
 ['$799,000'],
 ['$3.355 million (worldwide rentals)'],
 ['$3.275 million (worldwide rentals)'],
 ['$65 million'],
 ['$3.165 million (worldwide rentals)'],
 ['$2.56 million (worldwide rentals)'],
 ['$3.7 million (U.S. rental) + $575,000 (foreign rental)'],
 ['$1.625 million (worldwide rentals)'],
 ['$182 million'],
 ['$4.1 million (worldwide rentals)'],
 ['$2.4 million (1951, domestic)', '$3.5 million (1974, domestic)'],
 ['$2.1 million (US rentals)'],
 ['$87.4 million (United States and Canada)'],
 ['$1 million dollars (USA)'],
 ['$2.6 million (US/Canada rentals)'],
 ['$1,050,000'],
 ['$1.75 million (US and Canadian rentals)'],
 ['$28.2 million'],
 ['$2,150,000 (US)'],
 ['$187 million'],
 ['$2.1 million (US)'],
 ['$1.6 million (US)'],
 ['$1.7 

In [22]:
# Convert Budget and Box office to numbers
amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"

word_re = rf"\${number}(-|\sto\s|–)?({number})?\s({amounts})"
value_re = rf"\${number}"


def word_to_value(word):
    value_dict = {"thousand": 1000, "million": 1000000, "billion": 1000000000}
    return value_dict[word]


def parse_word_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    word = re.search(amounts, string, flags=re.IGNORECASE).group().lower()
    word_value = word_to_value(word)
    return value * word_value

def parse_value_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    return value

def money_conversion(money):
    if not money or money == "N/A":
        return None
    
    if isinstance(money, list):
        money = money[0]
        
    if not isinstance(money, str):
        return None

    word_syntax = re.search(word_re, money, flags=re.IGNORECASE)
    value_syntax = re.search(value_re, money)

    if word_syntax:
        return parse_word_syntax(word_syntax.group())

    elif value_syntax:
        return parse_value_syntax(value_syntax.group())
    else:
        return None

In [23]:
for movie in movie_info_list:
    movie["Budget (float)"] = money_conversion(movie.get("Budget") or "N/A")
    movie["Box office (float)"] = money_conversion(movie.get("Box office") or "N/A")

In [24]:
# Convert Dates into datetime object
[movie.get("Release date") or movie.get("Release dates") or "N/A" for movie in movie_info_list]

# MONTH-DAY-YEAR eg. July 21, 2017
# DAY-MONTH-YEAR eg. 21 July 2017
# AND FIRST ELEMENT IN LIST 

[['December 21, 1937 ( Carthay Circle Theatre )',
  'February 4, 1938 (United States)'],
 ['February 7, 1940 ( Center Theatre )', 'February 23, 1940 (United States)'],
 ['November 13, 1940'],
 ['June 27, 1941'],
 ['October 23, 1941 (New York City)', 'October 31, 1941 (U.S.)'],
 ['August 9, 1942 ( London )', 'August 13, 1942 (United States)'],
 ['August 24, 1942 (Rio de Janeiro)',
  'February 6, 1943 (Boston)',
  'February 19, 1943 (United States)'],
 ['July 17, 1943'],
 ['December 21, 1944 (Mexico City)', 'February 3, 1945 (U.S.)'],
 ['April 20, 1946 (New York City)', 'August 15, 1946 (United States)'],
 ['November 12, 1946 (Atlanta, premiere)', 'January 30, 1947 (Los Angeles)'],
 ['September 27, 1947'],
 ['May 27, 1948'],
 ['November 29, 1948 (Chicago)', 'January 19, 1949 (Indianapolis)'],
 ['October 5, 1949'],
 ['February 15, 1950 (Boston)', 'March 4, 1950 (United States)'],
 ['June 22, 1950 (London)', 'July 29, 1950 (United States)', 'July 19, 1950'],
 ['July 26, 1951 (London)',
  '

In [25]:
for movie in movie_info_list:
    if "Release dates" in movie:
        movie["Release date"] = movie.pop("Release dates")

In [26]:
def clean_date(date):
    return date.split("(")[0].strip()

def parse_month_day_year(date):
    fmts = ["%B %d, %Y", "%d %B %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(date, fmt)
        except ValueError:
            continue
    return None

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]
    
    if date == "N/A":
        return None
    
    date_string = clean_date(date)
    date_clear = parse_month_day_year(date_string)
    return date_clear

In [27]:
for movie in movie_info_list:
    movie["Release date (datetime)"] = date_conversion(movie.get("Release date") or "N/A")

In [28]:
movie_info_list[-5]

{'title': 'Sea Lions of the Galapagos',
 'Directed by': ['Hugh Wilson', 'Keith Scholey'],
 'Produced by': ['Hugh Wilson', 'Keith Scholey', 'Roy Conli'],
 'Narrated by': ['Brendan Fraser'],
 'Cinematography': ['Richard Wollocombe', 'Paul D. Stewart'],
 'Edited by': ['Andy Netley'],
 'Music by': ['Raphaelle Thibaut'],
 'Production companies': ['Disneynature', 'Silverback Films'],
 'Distributed by': ['Disney+'],
 'Release date': ['April 22, 2025'],
 'Running time': ['83 minutes'],
 'Country': ['United States'],
 'Language': ['English'],
 'Running time (int)': 83,
 'Budget (float)': None,
 'Box office (float)': None,
 'Release date (datetime)': datetime.datetime(2025, 4, 22, 0, 0)}

In [29]:
# convert datatime 
def _datetime_converter(o):
    if isinstance(o, datetime):
        return o.isoformat()
    raise TypeError(f"Type {type(o)} not serializable")

# save checkpoint to json
def save_checkpoint(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False, default=_datetime_converter)
    print(f"Checkpoint saved to {filename}")

# load checkpoint from json + convert datatime
def load_checkpoint(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
    for record in data:
        for key, value in record.items():
            if isinstance(value, str) and value.startswith(("19", "20")) and "T" in value:
                try:
                    record[key] = datetime.fromisoformat(value)
                except ValueError:
                    pass
    print(f"Checkpoint loaded from {filename}")
    return data

In [30]:
save_checkpoint(movie_info_list, "disney_data_scraped_checkpoint.json")

Checkpoint saved to disney_data_scraped_checkpoint.json


In [31]:
movie_info_list = load_checkpoint("disney_data_scraped_checkpoint.json")

Checkpoint loaded from disney_data_scraped_checkpoint.json


In [32]:
movie_info_list[-35]

{'title': "Chip 'n Dale: Rescue Rangers",
 'Directed by': ['Akiva Schaffer'],
 'Written by': ['Dan Gregor', 'Doug Mand'],
 'Based on': ["Chip 'n Dale: Rescue Rangers by Tad Stones Alan Zaslove",
  'Tad Stones',
  'Alan Zaslove',
  "Chip 'n' Dale by Bill Justice"],
 'Produced by': ['David Hoberman', 'Todd Lieberman'],
 'Starring': ['John Mulaney', 'Andy Samberg', 'KiKi Layne'],
 'Cinematography': ['Larry Fong'],
 'Edited by': ['Brian Olds'],
 'Music by': ['Brian Tyler', 'Mark Mueller', '(Theme Song)'],
 'Production companies': ['Walt Disney Pictures', 'Mandeville Films'],
 'Distributed by': ['Walt Disney Studios Motion Pictures'],
 'Running time': ['97 minutes'],
 'Country': ['United States'],
 'Language': ['English'],
 'Budget': ['~$70 million'],
 'Box office': ['$623,190'],
 'Running time (int)': 97,
 'Budget (float)': 70000000.0,
 'Box office (float)': 623190.0,
 'Release date': ['May 16, 2022 ( El Capitan Theatre )',
  'May 20, 2022 ( Disney+ )'],
 'Release date (datetime)': datetim

### Attach IMDB/Rotten Tomatoes/Metascore scores

In [33]:
# http://www.omdbapi.com/?apikey=[yourkey]&

In [34]:
load_dotenv()

def get_omdb_info(title):
    base_url = "http://www.omdbapi.com/?"
    parameters = {"apikey" : os.getenv("OMDB_API_KEY"), "t": title}
    params_encoded = urllib.parse.urlencode(parameters)
    full_url = base_url + params_encoded
    response = requests.get(full_url)
    return response.json()

def get_rotten_tomato(omdb_info):
    ratings = omdb_info.get("Ratings") or []
    for rating in ratings:
        if rating["Source"] == "Rotten Tomatoes":
            return rating["Value"]
    return None

def get_genre(omdb_info):
    genre_str = omdb_info.get("Genre") or ""
    if genre_str in ["", "N/A"]:
        return []
    return [g.strip() for g in genre_str.split(",")]

In [35]:
for index, movie in enumerate(movie_info_list):
    if index % 10 == 0:
        print(index)
    
    title = movie["title"]

    try:
        omdb_info = get_omdb_info(title)
        
        if omdb_info.get("Response") == "False":
            error_msg = omdb_info.get("Error", "Unknown error")

            if "Request limit reached" in error_msg:
                print(f"API limit reached at index {index} for '{title}'")
                break
            
            print(f"API error at index {index} for '{title}': {error_msg}")
            continue

        movie["imdb"] = omdb_info.get("imdbRating") or None
        movie["metascore"] = omdb_info.get("Metascore") or None
        movie["rotten_tomatos"] = get_rotten_tomato(omdb_info)
        movie["genres"] = get_genre(omdb_info)

    except requests.exceptions.RequestException as e:
        print(f"Network error while fetching '{title}': {e}")
    except Exception as e:
        print(f"Unexpected error for '{title}': {e}")

    time.sleep(1)

0
10
20
30
40
API error at index 43 for 'Toby Tyler or 10 Weeks with a Circus': Movie not found!
50
60
70
80
90
100
110
120
API error at index 129 for 'Escape from the Dark (The Littlest Horse Thieves)': Movie not found!
130
140
150
160
API error at index 165 for 'The Man from Snowy River II': Movie not found!
170
180
190
200
210
220
230
240
API error at index 246 for 'The Little Mermaid II: Return to the Sea': Movie not found!
250
API error at index 259 for 'Cinderella II: Dreams Come True': Movie not found!
260
API error at index 269 for '101 Dalmatians II:': Movie not found!
270
280
API error at index 286 for 'The Lion King 1½': Movie not found!
290
API error at index 292 for 'America's Heart and Soul': Movie not found!
300
API error at index 303 for 'Tarzan II': Movie not found!
310
API error at index 316 for 'Bambi II': Movie not found!
320
API error at index 328 for 'Cinderella III: A Twist in Time': Movie not found!
330
340
350
API error at index 359 for 'The Boys: The Sherman B

In [36]:
movie_info_list[0]

{'title': 'Snow White and the Seven Dwarfs',
 'Directed by': ['Perce Pearce',
  'William Cottrell',
  'Larry Morey',
  'Wilfred Jackson',
  'Ben Sharpsteen'],
 'Story by': ['Ted Sears',
  'Richard Creedon',
  'Otto Englander',
  'Dick Rickard',
  'Earl Hurd',
  'Merrill De Maris',
  'Dorothy Ann Blank',
  'Webb Smith'],
 'Based on': ['"', 'Snow White', '"', 'by the', 'Brothers Grimm'],
 'Produced by': ['Walt Disney'],
 'Music by': ['Frank Churchill', 'Leigh Harline', 'Paul Smith'],
 'Production company': ['Walt Disney Productions'],
 'Distributed by': ['RKO Radio Pictures'],
 'Running time': ['83 minutes'],
 'Country': ['United States'],
 'Language': ['English'],
 'Budget': ['$1.5 million'],
 'Box office': ['$418 million'],
 'Running time (int)': 83,
 'Budget (float)': 1500000.0,
 'Box office (float)': 418000000.0,
 'Release date': ['December 21, 1937 ( Carthay Circle Theatre )',
  'February 4, 1938 (United States)'],
 'Release date (datetime)': datetime.datetime(1937, 12, 21, 0, 0),
 

In [37]:
save_checkpoint(movie_info_list, "disney_data_scraped_final.json")

Checkpoint saved to disney_data_scraped_final.json


In [38]:
load_checkpoint("disney_data_scraped_final.json")

Checkpoint loaded from disney_data_scraped_final.json


[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['Perce Pearce',
   'William Cottrell',
   'Larry Morey',
   'Wilfred Jackson',
   'Ben Sharpsteen'],
  'Story by': ['Ted Sears',
   'Richard Creedon',
   'Otto Englander',
   'Dick Rickard',
   'Earl Hurd',
   'Merrill De Maris',
   'Dorothy Ann Blank',
   'Webb Smith'],
  'Based on': ['"', 'Snow White', '"', 'by the', 'Brothers Grimm'],
  'Produced by': ['Walt Disney'],
  'Music by': ['Frank Churchill', 'Leigh Harline', 'Paul Smith'],
  'Production company': ['Walt Disney Productions'],
  'Distributed by': ['RKO Radio Pictures'],
  'Running time': ['83 minutes'],
  'Country': ['United States'],
  'Language': ['English'],
  'Budget': ['$1.5 million'],
  'Box office': ['$418 million'],
  'Running time (int)': 83,
  'Budget (float)': 1500000.0,
  'Box office (float)': 418000000.0,
  'Release date': ['December 21, 1937 ( Carthay Circle Theatre )',
   'February 4, 1938 (United States)'],
  'Release date (datetime)': datetime.d

### Convert data to CSV

In [39]:
df = pd.DataFrame(movie_info_list)

In [40]:
df.head()

,title,Directed by,Story by,Based on,Produced by,Music by,Production company,Distributed by,Running time,Country,...,Traditional Chinese,Simplified Chinese,Literal meaning,Original title,French,Layouts by,Original language,Producers,Editors,Editor
0,Snow White and the Seven Dwarfs,"[Perce Pearce, William Cottrell, Larry Morey, ...","[Ted Sears, Richard Creedon, Otto Englander, D...","["", Snow White, "", by the, Brothers Grimm]",[Walt Disney],"[Frank Churchill, Leigh Harline, Paul Smith]",[Walt Disney Productions],[RKO Radio Pictures],[83 minutes],[United States],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pinocchio,"[Ben Sharpsteen, Hamilton Luske, Bill Roberts,...","[Ted Sears, Otto Englander, Webb Smith, Willia...","[The Adventures of Pinocchio, by, Carlo Collodi]",[Walt Disney],"[Leigh Harline, Paul J. Smith]",[Walt Disney Productions],[RKO Radio Pictures],[88 minutes],[United States],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fantasia,"[Samuel Armstrong, James Algar, Bill Roberts, ...","[Joe Grant, Dick Huemer]",NaN,"[Walt Disney, Ben Sharpsteen]",[See plot],[Walt Disney Productions],[RKO Radio Pictures],[126 minutes],[United States],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,The Reluctant Dragon,"[Alfred Werker, (live action), Hamilton Luske,...",NaN,NaN,[Walt Disney],"[Frank Churchill, Larry Morey]",[Walt Disney Productions],[RKO Radio Pictures],[74 minutes],[United States],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dumbo,"[Ben Sharpsteen, Norman Ferguson, Wilfred Jack...","[Joe Grant, Dick Huemer]","[Dumbo, the Flying Elephant, by, Helen Aberson...",[Walt Disney],"[Frank Churchill, Oliver Wallace]",[Walt Disney Productions],[RKO Radio Pictures],[64 minutes],[United States],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df.to_csv("disney_movie_scraped_final.csv")

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 54 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   title                    536 non-null    object        
 1   Directed by              534 non-null    object        
 2   Story by                 170 non-null    object        
 3   Based on                 303 non-null    object        
 4   Produced by              524 non-null    object        
 5   Music by                 532 non-null    object        
 6   Production company       287 non-null    object        
 7   Distributed by           533 non-null    object        
 8   Running time             533 non-null    object        
 9   Country                  466 non-null    object        
 10  Language                 513 non-null    object        
 11  Budget                   328 non-null    object        
 12  Box office               414 non-nul